In [24]:
import sympy as sym
from sympy import Array
import numpy as np
import plotly.graph_objects as go
import plotly.express as plx

plx.defaults.width = 600
plx.defaults.height = 600

In [25]:
i, x, y, px, py = sym.symbols("i, x, y, p_x, p_y", real=True)
delta = sym.symbols("\delta", real=True)
L = sym.symbols("L", real=True)

X = Array([x, y])
P = Array([px, py])

H = -sym.sqrt((1 + delta) ** 2 - px**2 - py**2)


In [26]:
def poisson_brackets(f, g, X, P):
    return sum([f.diff(xi) * g.diff(pi) - f.diff(pi) * g.diff(xi) for xi, pi in zip(X, P)])


In [81]:
def lie_transformation(f, g, X, P, order=4):
    # Special case for i = 0 as transform is always identity
    poisson_transform = g
    transform = poisson_transform 
    for i in range(1, order, 1):
        poisson_transform = poisson_brackets(f, poisson_transform, X, P)
        transform += poisson_transform / sym.factorial(i)    
        if poisson_transform == 0:
            print("Exact transformation found at order", i)
            return sym.simplify(transform)
    print("Exact transformation could not be found up to order", order)
    return sym.simplify(transform)



def lie_transformation_hamiltonian_4D(X, P, H, L, order=2, yoshida_coeff = 1.):
    x_new, y_new, px_new, py_new = [
        lie_transformation(-L * yoshida_coeff * H, g, X, P, order) for g in X.tolist() + P.tolist()
    ]
    return x_new, px_new, y_new, py_new


In [82]:
def generate_hamiltonian(order_magnet, delta, X, P, skew = False, k = None, rho = None, approx = True, return_separate_hamiltonians = False):
    if k is None:
        k = sym.Symbol("k"+'_' + str(order_magnet), real=True)
    if rho is None:
        rho = sym.Symbol("rho", real=True)

    # Take into account if skew components are present
    if skew:
        fact_norm = 0
        fact_skew = 1j
    else:
        fact_norm = 1
        fact_skew = 0
    
    # Drift is a special case
    if order_magnet == -1:
        if approx:
            Hd = (P[0]**2 + P[1]**2)/(2*(1+delta))
            Hk = 0
            H = Hd + Hk
        else:
            H = -((1 + delta) ** 2 - px**2 - py**2)**0.5
            if return_separate_hamiltonians:
                print('No separate hamiltonians available for exact drift')
                return sym.simplify(H)
            
    # Dipole is also a special case as Bx is not symmetrical with By, and k = 1/rho
    elif order_magnet == 0:
        Hd = (P[0]**2 + P[1]**2)/(2*(1+delta))
        Hk = x*delta/rho +(x**2)/(2*rho**2)
        H = Hd + Hk

    # Multipoles
    else:
        Hd = (P[0]**2 + P[1]**2)/(2*(1+delta))
        Hk = 1/(1+order_magnet) * sym.re( (fact_norm*k + fact_skew*k) * (X[0]+1j*X[1])**(order_magnet+1))
        H = Hd + Hk

    if return_separate_hamiltonians:
        return sym.simplify(Hd), sym.simplify(Hk)
    else:
        return sym.simplify(H)


H = generate_hamiltonian(1, delta, X, P, approx = False)
H

0.5*k_1*(x**2 - 1.0*y**2) + 0.5*p_x**2 + 0.5*p_y**2

In [83]:
x_new, px_new, y_new, py_new = lie_transformation_hamiltonian_4D(X, P, H, L, order =5)


Exact transformation could not be found up to order 5
Exact transformation could not be found up to order 5
Exact transformation could not be found up to order 5
Exact transformation could not be found up to order 5


In [88]:
def yoshida_coeff_calculator(order):
    if order%2!=0:
        raise("Yoshida coefficients can only be computed for even orders.") 
    
    S = [0.5, 1, 0.5]
    for n in range(1, int(order/2 )):
        alpha = 2.**(1./(2*n+1))
        x0=- alpha/(2.-alpha)
        x1=1/(2.0-alpha)
        TC=[i*x0 for i in S]
        TL=[i*x1 for i in S]
        T=[]
        for i in TL [ : - 1]:
            T.append(i)
        T.append(TL[-1]+TC [ 0 ])
        for i in TC[ 1 : - 1]:
            T.append(i)
        T.append(TC[-1]+TL [ 0 ])
        for i in TL [ 1 : ]:
            T.append(i)
        S=T
    return S
                         

def symplectic_analytical_integrator(order_integrator, Hd, Hk, X, P, L, max_order_lie_transform = 20):
    S = yoshida_coeff_calculator(order_integrator)
    l_transforms = []
    for idx, coeff in enumerate(S):
        if idx%2 == 0:
            x_new, px_new, y_new, py_new = lie_transformation_hamiltonian_4D(X, P, Hd, L, order =max_order_lie_transform , yoshida_coeff = coeff)
            l_transforms.append([x_new, px_new, y_new, py_new])
        else:
            x_new, px_new, y_new, py_new = lie_transformation_hamiltonian_4D(X, P, Hk, L, order =max_order_lie_transform , yoshida_coeff = coeff)
            l_transforms.append([x_new, px_new, y_new, py_new])
    return l_transforms

def symplectic_numerical_integrator(l_transforms)
        
1 f o r i i n c o e f f :
2 i f c o u n t e r%2 == 0 :
3 Hd_numeric_x = sympy . l ambd i f y ( ( x , p ) , l i e_ t r a n s f o rm (−L∗ i ∗Hd , x
) , numpy )
4 Hd_numeric_p = sympy . l ambd i f y ( ( x , p ) , l i e_ t r a n s f o rm (−L∗ i ∗Hd , p
) , numpy )
5 e l l i p s e = l i e_ t r a n s f o rm (L∗ i ∗Hd , e l l i p s e )
6 x f = Hd_numeric_x ( x i n i t , p i n i t )
7 p f = Hd_numeric_p ( x i n i t , p i n i t )
8 e l s e :
9 Hk_numeric_x = sympy . l ambd i f y ( ( x , p ) , l i e_ t r a n s f o rm (−L∗ i ∗Hk , x
) , numpy )
10 Hk_numeric_p = sympy . l ambd i f y ( ( x , p ) , l i e_ t r a n s f o rm (−L∗ i ∗Hk , p
) , numpy )
11 e l l i p s e = l i e_ t r a n s f o rm (L∗ i ∗Hk , e l l i p s e )
12 x f = Hk_numeric_x ( x i n i t , p i n i t )
13 p f = Hk_numeric_p ( x i n i t , p i n i t )
14
15 x i n i t = x f
16 p i n i t = p f
17 c o u n t e r = c o u n t e r + 1

In [89]:
yoshida_coeff_calculator(8)

[0.8858166925977682,
 1.7716333851955364,
 -0.23024240475441596,
 -2.2321181947043685,
 -0.23024240475441596,
 1.7716333851955364,
 -0.13171948501820255,
 -2.035072355231941,
 0.26447907159195916,
 2.5640304984158595,
 0.26447907159195916,
 -2.035072355231941,
 -0.13171948501820255,
 1.7716333851955364,
 -0.23024240475441596,
 -2.2321181947043685,
 -0.23024240475441596,
 1.7716333851955364,
 -0.09220422873664647,
 -1.9560418426688293,
 0.2542082246923922,
 2.4644582920536138,
 0.2542082246923922,
 -1.9560418426688293,
 0.14543010215511223,
 2.246902046979054,
 -0.29200856953086757,
 -2.8309191860407887,
 -0.29200856953086757,
 2.246902046979054,
 0.14543010215511223,
 -1.9560418426688293,
 0.2542082246923922,
 2.4644582920536138,
 0.2542082246923922,
 -1.9560418426688293,
 -0.09220422873664647,
 1.7716333851955364,
 -0.23024240475441596,
 -2.2321181947043685,
 -0.23024240475441596,
 1.7716333851955364,
 -0.13171948501820255,
 -2.035072355231941,
 0.26447907159195916,
 2.564030498415859

In [75]:
# Define a set of particles with given position and momenta
n_particles = 500
x_array = np.random.normal( 0. , 10**(-3), size =n_particles)
#px_array = np.random.normal (0. , 10**(-3), size =n_particles)
px_array = np.random.normal (0. , 10**(-4), size =n_particles)

# Compute the emittance 
std_x = x_array.std()
std_px = px_array.std()
cov_xpx = np.nanmean( (x_array-np.mean(x_array)) * (px_array - np.mean(px_array)))
emittance = (std_x**2*std_px**2-cov_xpx)**0.5

# Compute the Twiss parameters
alpha = -cov_xpx / emittance
beta = std_x**2/ emittance
gamma = std_px**2/ emittance

# Compute the corresponding ellipse in the phase-spae
x_ellipse = np.linspace(np.min(x_array) , np.max(x_array), 100)
p_ellipse = np.linspace(np.min(px_array) , np.max(px_array), 100)
x_ellipse, p_ellipse = np.meshgrid (x_ellipse, p_ellipse)
ellipse = gamma * x_ellipse**2+2*alpha*x_ellipse * p_ellipse+beta*p_ellipse**2-emittance


/tmp/ipykernel_837729/2664950794.py:11: RuntimeWarning:

invalid value encountered in double_scalars



In [76]:
# Define colors according to PCA projection in 1D
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca.fit(np.array([x_array, px_array]).T)
pca_projection = pca.transform(np.array([x_array, px_array]).T)

# Rescale pca projection to be between 0 and 1
pca_projection = (pca_projection - np.min(pca_projection))/(np.max(pca_projection)-np.min(pca_projection))

pca_projection = np.squeeze(pca_projection)

In [77]:
# Plot points and ellipse
fig = plx.scatter(x=x_array, y=px_array, labels={'x':r'$x$', 'y':r'$p_x$'}, color_continuous_scale='Turbo', color=pca_projection, opacity=0.8)
#fig.add_trace(go.Contour(x=x_ellipse[0,:], y=p_ellipse[:,0], z=ellipse, colorscale='Blues', showscale=False, opacity=0.5))
fig.update_layout(yaxis_range=[-0.005,0.005])
fig.update_layout(xaxis_range=[-0.005,0.005])

fig.show()

In [78]:
# Transport the beam through a drift space of length 2. with no dispersion
L = 2.
delta = 0.
H = generate_hamiltonian(-1, delta, X, P, k =None)
x_new, px_new, y_new, py_new = lie_transformation_hamiltonian_4D(X, P, H, L, order=15)

# # Evaluate the result with lambdify
f_x = sym.lambdify((x, px) , x_new)
f_px = sym.lambdify((x, px) , px_new)
x_array_transformed = f_x(x_array, px_array)
px_array_transformed = f_px(x_array, px_array)


Exact transformation found at order 2
Exact transformation found at order 2
Exact transformation found at order 1
Exact transformation found at order 1


In [79]:
# Plot points and ellipse
fig = plx.scatter(x=x_array_transformed, y=px_array_transformed, labels={'x':r'$x$', 'y':r'$p_x$'}, color_continuous_scale='Turbo', color=pca_projection, opacity=0.8)
fig.update_layout(yaxis_range=[-0.005,0.005])
fig.update_layout(xaxis_range=[-0.005,0.005])
#fig.add_trace(go.Contour(x=x_ellipse[0,:], y=p_ellipse[:,0], z=ellipse, colorscale='Blues', showscale=False, opacity=0.5))
fig.show()

In [80]:
# Now transport the beam through a quadrupole of length 0.2 and strength 0.5, with no dispersion
for L in np.linspace(0.00,15,20):
    L = L
    delta = 0.
    k_1 = 0.1
    H = generate_hamiltonian(2, delta, X, P, k =k_1)
    x_new, px_new, y_new, py_new = lie_transformation_hamiltonian_4D(X, P, H, L, order=50)

    # Redefine variables as check
    #x_new = sym.cos(k_1**0.5*L)*x + sym.sin(k_1**0.5*L)/k_1**0.5*px
    #px_new = -k_1**0.5*sym.sin(k_1**0.5*L)*x + sym.cos(k_1**0.5*L)*px

    # # Evaluate the result with lambdify
    f_x = sym.lambdify((x, px) , x_new)
    f_px = sym.lambdify((x, px) , px_new)
    x_array_transformedd = f_x(x_array, px_array)
    px_array_transformedd = f_px(x_array, px_array)

    # Plot points and ellipse
    fig = plx.scatter(x=x_array_transformedd, y=px_array_transformedd, labels={'x':r'$x$', 'y':r'$p_x$'}, color_continuous_scale='Turbo', color=pca_projection, opacity=0.8, title='L = '+str(L))
    fig.update_layout(yaxis_range=[-0.01,0.01])
    fig.update_layout(xaxis_range=[-0.01,0.01])
    fig.update_coloraxes(showscale=False)
    #fig.add_trace(go.Contour(x=x_ellipse[0,:], y=p_ellipse[:,0], z=ellipse, colorscale='Blues', showscale=False, opacity=0.5))
    fig.show()

Exact transformation found at order 1
Exact transformation found at order 1
Exact transformation found at order 1
Exact transformation found at order 1


KeyboardInterrupt: 